In [ ]:
import cv2
import math
import cvzone
from ultralytics import YOLO
import supervision as sv
from sort import Sort
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model = YOLO('./yolov10/weights/yolov8n.pt') 

In [ ]:
img = './KOA_Nassau_2697x1517.jpg'
img = cv2.imread(img)
result = model(img)[0]
detection = sv.Detections.from_ultralytics(result)

In [ ]:
box_anonator = sv.BoxAnnotator()
anonate = box_anonator.annotate(
    scene=img,
    detections=detection
)

In [ ]:
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',anonate)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
labels = [
    f'{className} {confidenece}'
    for className,confidenece in zip(detection['class_name'], detection.confidence)
]

labelAnonter = sv.LabelAnnotator(text_position=sv.Position.CENTER)
annotated_frame = labelAnonter.annotate(
    scene=img,
    detections=detection,
    labels=labels
)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',annotated_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
video_info = sv.VideoInfo.from_video_path('./Shopping.mp4')
track = sv.ByteTrack(frame_rate=video_info.fps)
track

In [ ]:
cap = cv2.VideoCapture('./Shopping.mp4')

detections_for_tracking = []
total_people_count = 0
while True:
    res,frame = cap.read()
    result = model(frame)[0]
    detection = sv.Detections.from_ultralytics(result)
    box_anonator = sv.EllipseAnnotator()

    path_to_video = './Shopping.mp4'
    video_info = sv.VideoInfo.from_video_path(path_to_video)
    byte_tracker = sv.ByteTrack(frame_rate=video_info.fps)
    frame_generator = sv.get_video_frames_generator(path_to_video)
    fram = next(frame_generator)
    tracked_detections = byte_tracker.update_with_detections(detection)

    people_count = sum(1 for label in detection['class_name'] if label == 'person')
    total_people_count += people_count
    text = f'Frame: {total_people_count}'
    frame = cv2.putText(frame, f'{people_count}', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=2)
    
    
    
    anonate = box_anonator.annotate(
            scene=frame,
            detections=detection
        )
    labels = [
    f'{className} {confidenece:.2f}'
        for className,confidenece in zip(detection['class_name'], detection.confidence)
    ]

    labelAnonter = sv.LabelAnnotator(text_position=sv.Position.CENTER)
    anonate = labelAnonter.annotate(
        scene=frame,
        detections=detection,
        labels=labels
    )
    cv2.namedWindow('example',cv2.WINDOW_NORMAL)
    cv2.imshow('example',anonate)
    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import supervision as sv

# Initialize video capture
cap = cv2.VideoCapture('./Shopping.mp4')

# Initialize variables
unique_ids = set()  # To keep track of unique person IDs
total_people_count = 0


box_anonator = sv.EllipseAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)
path_to_video = './Shopping.mp4'
video_info = sv.VideoInfo.from_video_path(path_to_video)
byte_tracker = sv.ByteTrack(frame_rate=video_info.fps)
frame_generator = sv.get_video_frames_generator(path_to_video)

fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can use other codecs as well
out = cv2.VideoWriter('./Annotated_Shopping.mp4', fourcc, video_info.fps, (int(cap.get(3)), int(cap.get(4))))

while True:
    res, frame = cap.read()
    if not res:
        break

    result = model(frame)[0]
    detection = sv.Detections.from_ultralytics(result)
    
    # Track detections
    tracked_detections = byte_tracker.update_with_detections(detection)

    # Assuming tracked_detections is a list of tuples (id, class_id)
    fpeople_count = sum(1 for label in detection['class_name'] if label == 'person')
    total_people_count += people_count
    text = f'Frame: {total_people_count}'
    frame = cv2.putText(frame, f'{people_count}', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 0), thickness=2)
    
    labels = [
        f'{className} {confidence:.2f}'
        for className, confidence in zip(detection['class_name'], detection.confidence)
    ]
    
    annotated_frame = box_anonator.annotate(scene=frame, detections=detection)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detection, labels=labels)
    
    # Write the annotated frame to the output video
    out.write(annotated_frame)
    
    cv2.namedWindow('example', cv2.WINDOW_NORMAL)
    cv2.imshow('example', annotated_frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Release the VideoWriter object
cv2.destroyAllWindows()

# Print total number of unique people detected
print(f'Total number of unique people detected: {total_people_count}')
